In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf

rng = np.random.RandomState(0)

In [5]:
# Initialisierung von DataFrames und Data Cleaning
df = pd.read_csv('Laptop-Preise.csv', sep=';', decimal=',')
df = df[df.extern_Schnittstellen != 2300] # Ausreißer löschen
df = df.reset_index(drop=True) # Reset Index

# Nur die 5 Spalten auswählen, die auch den größten Erklärungsgehalt haben
selected_columns = ['Preis', 'Akku_Kapazitaet', 'Arbeitsspeicher', 'Kerne', 'Mobilfunk_vorhanden', 'SSD']

# Create a new DataFrame with only the selected columns
df = df[selected_columns]

df_noPrice = df.drop('Preis', axis=1)
imputed_simul_knn = pd.DataFrame(columns=['MSE KNN_1', 'MSE KNN_3', 'MSE KNN_5', 'SE KNN_1', 'SE KNN_3', 'SE KNN_5'])
imputed_stats_knn = pd.DataFrame(columns=['MSE KNN_1', 'MSE KNN_3', 'MSE KNN_5', 'SE KNN_1', 'SE KNN_3', 'SE KNN_5'], index = np.arange(0.1, 1, 0.1))

imputed_simul_ols = pd.DataFrame(columns=['MSE OLS', 'SE OLS'])
imputed_stats_ols = pd.DataFrame(columns=['MSE OLS', 'SE OLS'], index = np.arange(0.1, 1, 0.1))

# Skalierung (Standardisierung) von df_noPrice
col_names = df_noPrice.columns
scaler = StandardScaler().fit(df_noPrice.values)
df_noPrice = scaler.transform(df_noPrice.values)
df_noPrice = pd.DataFrame(df_noPrice, columns=col_names)

# Skalierung (Standardisierung) von df mit Preis (Preis ist unverändert)
df_std = df_noPrice.copy()
df_std.insert(0, 'Preis', df['Preis'])

In [7]:
# OLS Model 
# fitting the model 
# df_noPrice ist bereits standardisiert
model = sm.OLS(df['Preis'], sm.add_constant(df_noPrice)).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Preis   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.788
Method:                 Least Squares   F-statistic:                     771.4
Date:                Sat, 13 Jan 2024   Prob (F-statistic):               0.00
Time:                        19:02:28   Log-Likelihood:                -7534.6
No. Observations:                1038   AIC:                         1.508e+04
Df Residuals:                    1032   BIC:                         1.511e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                1333.9633    

In [8]:
train_values, test_values, train_labels, test_labels = train_test_split(df_noPrice, df['Preis'], test_size=0.01)

def del_ran(df_exog, labels, chance):
    rand_array = np.random.rand(df_exog.shape[0])
    delete_entries = rand_array < chance
    keep_entries = rand_array >= chance
    
    return [df_exog[delete_entries], labels[delete_entries], df_exog[keep_entries], labels[keep_entries]]

temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = 0.1)

In [9]:
def impute_ols(test_values, test_labels, train_values, train_labels):
    
    # OLS Model
    # fitting the model 
    model = sm.OLS(train_labels, sm.add_constant(train_values)).fit() 

    imputed_values = model.predict(exog = sm.add_constant(test_values, has_constant='add')).tolist()
    return [np.mean((imputed_values-test_labels)**2), stats.sem(list(train_labels) + imputed_values)]

In [10]:
def impute_knn(test_values, test_labels, train_values, train_labels):
    
    tree = KDTree(train_values.values, leaf_size=5)

    imputed_values_knn_1 = []
    imputed_values_knn_3 = []
    imputed_values_knn_5 = []

    for index, entry in enumerate(test_values.values):
 
        dist, ind = tree.query([entry], k=5)
        ind = ind[0]

        current_impute_knn_1 = np.mean(train_labels.values[ind][0])
        current_impute_knn_3 = np.mean(train_labels.values[ind][:3])
        current_impute_knn_5 = np.mean(train_labels.values[ind])

        imputed_values_knn_1.append(current_impute_knn_1)
        imputed_values_knn_3.append(current_impute_knn_3)
        imputed_values_knn_5.append(current_impute_knn_5)
       
        # print(train_labels.values[ind])
        # print(current_impute_knn_1//1, current_impute_knn_3//1, current_impute_knn_5//1)
        # print(test_labels.values[index])
        # print(train_values.values[ind])

    mse_knn_1 = np.mean((test_labels.values - imputed_values_knn_1)**2)
    mse_knn_3 = np.mean((test_labels.values - imputed_values_knn_3)**2)
    mse_knn_5 = np.mean((test_labels.values - imputed_values_knn_5)**2)

    sem_knn_1 = stats.sem(list(train_labels.values)+imputed_values_knn_1)
    sem_knn_3 = stats.sem(list(train_labels.values)+imputed_values_knn_3)
    sem_knn_5 = stats.sem(list(train_labels.values)+imputed_values_knn_5)


    return [mse_knn_1, mse_knn_3, mse_knn_5, sem_knn_1, sem_knn_3, sem_knn_5]

In [11]:
def simul_knn():
    for c in np.arange(0.1, 1, 0.1):

        for i in tqdm(range(1000)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_knn.at[i] = impute_knn(temp[0], temp[1], temp[2], temp[3])

        imputed_stats_knn.loc[c] = imputed_simul_knn.mean()
    imputed_stats_knn

simul_knn()
    

100%|██████████| 1000/1000 [01:36<00:00, 10.33it/s]


In [12]:
def simul_ols():
    for c in np.arange(0.1, 1, 0.1):

        for i in tqdm(range(1000)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_ols.at[i] = impute_ols(temp[0], temp[1], temp[2], temp[3])
        imputed_stats_ols.loc[c] = imputed_simul_ols.mean()
    imputed_stats_ols

simul_ols()

100%|██████████| 1000/1000 [00:05<00:00, 172.61it/s]


Bei der Simulation wird irgendwann ein Fehler geworfen. Das liegt daran, dass, wenn zu viele Einträge gelöscht werden, die Matrix ihren vollen Rang verliert. Einige Dummy-Variablen sind relativ selten. Wenn nun alle Einträge mit der Dummy-Variable auf 1 (oder 0) gelöscht werden, dann wird wegen der linearen Abhängigkeit zur Konstante diese Variable gelöscht - so vermeiden wir Kollinarität.
Wenn dann aber nun im Test-Datensatz die Variable in ihrer Ausprägung 0 (oder 1) besitzt, dann können haben wir dafür keinen passenden Regressionskoeffizienten. Es werden dann 33 Variablen als Input geliefert, obwohl wir nur 32 Regressionskoeffizienten haben. Anschließend wird eine Fehlermeldung geworfen, dass der Eintrag, den wir vorhersagen wollen die falsche shape hat (32 statt 33). 
Es stellt sich zudem an diesem Punkt außerdem die Frage, wie sinnvoll es ist Variablen imputieren zu wollen, bei denen 70% der Einträge fehlen.

Bei den Auswertungen ist zu beachten, dass der wahre Standardfehler von Preis bei 23.2318 liegt.
Wir können also deutlich erkennen, dass jede Imputation der fehlenden Werte den Standardfehler künstlich verringert.

Es ist auch relativ deutlich klar, dass ein höherer k-Wert für KNN dazu führt, dass der Standardfehler weiter sinkt. Das liegt daran, dass bei einem höheren k-Wert der Mittelwert von einem größeren Teil des Datensatzes genommen wird. Dadurch wird der Mittelwert (oder zumindest Mittelwert-nahe Werte) imputiert.

In [13]:
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1)

,MSE OLS,SE OLS,MSE KNN_1,MSE KNN_3,MSE KNN_5,SE KNN_1,SE KNN_3,SE KNN_5
0.1,119245.735104,22.984699,116770.142579,95203.44479,94921.032905,23.32205,23.214856,23.19374
0.2,120033.382944,22.730275,124249.139637,98554.959703,97555.710257,23.434547,23.178803,23.121627
0.3,120302.551646,22.491758,128478.170649,100163.602806,98160.465573,23.517282,23.099106,22.989855
0.4,120688.517929,22.24438,131191.476558,102606.077241,100290.433118,23.559607,22.989658,22.819185
0.5,120977.103493,22.009073,135937.099852,106143.580781,103855.640808,23.598222,22.847831,22.62551
0.6,121639.69477,21.715219,141867.097612,110421.655814,108926.183917,23.559771,22.606785,22.316978
0.7,121952.115228,21.485907,150205.020485,117145.480998,116295.654131,23.500937,22.312434,21.97279
0.8,123486.915217,21.220231,164057.651964,128669.002791,127585.690546,23.30574,21.903111,21.418192
0.9,129019.769876,21.049056,193353.307628,152856.653519,153180.733529,22.985484,20.997723,19.923487
